In [ ]:
!pip list

In [ ]:
! git clone https://github.com/skumra/robotic-grasping.git

Cloning into 'robotic-grasping'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 653 (delta 0), reused 1 (delta 0), pack-reused 648
Receiving objects: 100% (653/653), 39.10 MiB | 26.10 MiB/s, done.
Resolving deltas: 100% (392/392), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d oneoneliu/cornell-grasp

100% 4.84G/4.85G [01:37<00:00, 43.1MB/s]
100% 4.85G/4.85G [01:38<00:00, 53.1MB/s]


# Preparation

In [ ]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/GR_ConvNet_Code')
!pwd

/content
/content/drive/MyDrive/GR_ConvNet_Code


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 124 kB 31.7 MB/s 
     |████████████████████████████████| 16.4 MB 258 kB/s 
     |████████████████████████████████| 29.6 MB 115 kB/s 
     |████████████████████████████████| 78 kB 9.2 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 636 kB 56.8 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=a2c2716ccaed638e44b89bff3d4a92daf2f751c46313d5900a187debc1387938
  Stored in directory: /root/.cache/pip/wheels/e6/67/af/f1ad15974b8fd95f59a63dbf854483ebe5c7a46a93930798b8
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210924-py3-none-any.whl size=60829 sha256=6228b129627a0585ede1d21d54e656d7386b57d0cdce37ff0252e39cbd2e1475
  Stored in directory: /root/.cache/pip/wheels/70/c6/de/aa41c65141bdbc9a8aa4b303ce26482aa2f1720ff41b7f17c3
Successfully built gpustat fvcore
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling 

In [ ]:
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/cornell-grasp.zip -d /content/cornell_dataset

Archive:  /content/drive/MyDrive/Grasp_Detection_Workplace/cornell-grasp.zip
  inflating: /content/cornell_dataset/01/pcd0100.txt  
  inflating: /content/cornell_dataset/01/pcd0100cneg.txt  
  inflating: /content/cornell_dataset/01/pcd0100cpos.txt  
  inflating: /content/cornell_dataset/01/pcd0100d.tiff  
  inflating: /content/cornell_dataset/01/pcd0100r.png  
  inflating: /content/cornell_dataset/01/pcd0101.txt  
  inflating: /content/cornell_dataset/01/pcd0101cneg.txt  
  inflating: /content/cornell_dataset/01/pcd0101cpos.txt  
  inflating: /content/cornell_dataset/01/pcd0101d.tiff  
  inflating: /content/cornell_dataset/01/pcd0101r.png  
  inflating: /content/cornell_dataset/01/pcd0102.txt  
  inflating: /content/cornell_dataset/01/pcd0102cneg.txt  
  inflating: /content/cornell_dataset/01/pcd0102cpos.txt  
  inflating: /content/cornell_dataset/01/pcd0102d.tiff  
  inflating: /content/cornell_dataset/01/pcd0102r.png  
  inflating: /content/cornell_dataset/01/pcd0103.txt  
  inflatin

In [ ]:
filepath='/content/drive/MyDrive/GR_ConvNet_Code/logs'
if not os.path.exists(filepath):
    os.mkdir(filepath)

# Transfer to .tiff Depth Images

In [ ]:
%%writefile /content/drive/MyDrive/GR_ConvNet_Code/utils/dataset_processing/generate_cornell_depth2.py
import argparse
import glob
import os

import numpy as np
from imageio import imsave

from utils.dataset_processing.image import DepthImage

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Generate depth images from Cornell PCD files.')
    #parser.add_argument('path', type=str, help='Path to Cornell Grasping Dataset')
    args = parser.parse_args()

    pcd = '/content/drive/MyDrive/cornell_grasp_dataset/02/pcd0223.txt'
    di = DepthImage.from_pcd(pcd, (480, 640))
    di.inpaint()
    of_name = '../pcd0223test.tiff'
    print(of_name)
    imsave(of_name, di.img.astype(np.float32))

Overwriting /content/drive/MyDrive/GR_ConvNet_Code/utils/dataset_processing/generate_cornell_depth2.py


In [ ]:
!python -m utils.dataset_processing.generate_cornell_depth2

../pcd0223test.tiff


# Check cuda

In [ ]:
import torch
# 判断cuda是否可用；
print(torch.cuda.is_available())
# 获取gpu数量；
print(torch.cuda.device_count())
# 获取gpu名字；
print(torch.cuda.get_device_name(0))
# 返回当前gpu设备索引，默认从0开始；
print(torch.cuda.current_device())
# 查看tensor或者model在哪块GPU上
print(torch.tensor([0]).get_device())

True
1
Tesla T4
0
-1


In [ ]:
!cat /proc/cpuinfo 
!nvidia-smi

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

# Demo

In [ ]:
from utils.data import get_dataset
import torch
import torch.utils.data
import numpy as np

Dataset = get_dataset("cornell")
dataset = Dataset('/content/cornell_dataset',
                      ds_rotate=0.0,
                      random_rotate=True,
                      random_zoom=True,
                      include_depth=1,
                      include_rgb=1)

indices = list(range(dataset.length))
print(indices)

indices = indices[int(dataset.length*0.2):] + indices[:int(dataset.length*0.2)]
print(indices)
print(len(indices))
print(indices[-1])
print('--------------')

split = int(np.floor(0.8 * dataset.length))
# if True:
        # np.random.seed(123)
        # np.random.shuffle(indices)
train_indices, val_indices = indices[:split], indices[split:]
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_indices)
print(train_indices)
print('After reshuffling at every epoch...')

# np.random.seed(None)
# np.random.shuffle(train_indices)
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)

train_data = torch.utils.data.DataLoader(
    dataset,
    batch_size=8,
    num_workers=8,
    sampler=train_sampler
    # shuffle=False
)
val_data = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    num_workers=8,
    sampler=val_sampler
)

print(type(train_indices))
print(type(train_sampler))
print(type(train_data))
print(train_indices)
print(train_sampler)
print(train_sampler.indices)
print(train_data)
print(train_data.sampler.indices)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [ ]:
# make a unlimited generator demo
def yes_or_no():
    while True:
        yield "yes"
        yield "no"
g=yes_or_no()
for i in range(20):
    print(next(g))

Tasks:

1. If the speed is improved by setting the params -num-workers 20 --batch-size 16 --batches-per-epoch 500 ? -  *not evident*
2. If the network is trained better (detection accuracy, the trend of loss) when the params are set as above ? -  

**Optimal Performance:** The best performance has reached **0.955** without any shuffling operation.   
**batchsize=16**, channel_size=32, **learning rate=0.001**, use rgbd, split=0.9  

Results are saved in /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell


**Another Optimal Performance** has reached **0.966**
batchsize=8, channel_size=32, learning rate=0.001, use rgbd, split=0.9  
Results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell    

**Note that**      
*  no shuffling, 
*  the validation set contains only 89 images 
*  the original training set is larger than ones in 5-fold validation  




    
    

In [ ]:
!python train_network.py --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 20 --batch-size 16 --batches-per-epoch 500

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 796
root        : INFO     Validation size: 89
root        : INFO     Done
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d-2         [-1, 32, 224, 224]              64
            Conv2d-3         [-1, 64, 112, 112]          32,832
       BatchNorm2d-4         [-1, 64, 112, 112]             128
            Conv2d-5          [-1, 128, 56, 56]         131,200
       BatchNorm2d-6          [-1, 128, 56, 56]             256
            Conv2d-7          [-1, 128, 56, 56]         147,584
       BatchNorm2d-8          [-1, 128, 56, 56]             256
            Co

In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 89
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96
INFO:root:Average evaluation time per image: 100.25539558924986ms
INFO:root:IOU Results: 85/89 = 0.955056


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --ds-rotate 520 --augment --ds-shuffle --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96
INFO:root:Average evaluation time per image: 102.03508307031318ms
INFO:root:IOU Results: 816/885 = 0.922034


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --augment --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96
INFO:root:Average evaluation time per image: 99.31852642425709ms
INFO:root:IOU Results: 828/885 = 0.935593


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210323_0421_training_cornell/epoch_24_iou_0.96
INFO:root:Average evaluation time per image: 95.27233738010213ms
INFO:root:IOU Results: 764/885 = 0.863277


Conclusions of the 4 evaluation experiments above:

1. Augmentation before evaluating the whole data will improve the detection accuracy. -  **why ?**
2. when evaluate the network, the augmented dataset performs better than the original one. However, the augmented images are not counted in the validation set, so does the training proces. - **why ?**
3. The results of evaluate.py in evaluate process performs the same with utils/dataset_processing/evaluation.py in training process. - **But what are the differences ?**
    Why there is not net.eval() in evaluate.py?
4. Why the batch_size of test set in evaluation segment should only be assigned to 1?


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell/epoch_13_iou_0.97 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 89
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell/epoch_13_iou_0.97
INFO:root:Average evaluation time per image: 104.38177826699247ms
INFO:root:IOU Results: 86/89 = 0.966292


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell/epoch_13_iou_0.97 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --ds-rotate 520 --augment --ds-shuffle --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell/epoch_13_iou_0.97
INFO:root:Average evaluation time per image: 99.10004502635891ms
INFO:root:IOU Results: 742/885 = 0.838418


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell/epoch_13_iou_0.97 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --split 0

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell/epoch_13_iou_0.97
INFO:root:Average evaluation time per image: 94.51917174172267ms
INFO:root:IOU Results: 708/885 = 0.800000


Tasks:
 

1. logging infomation added  √
2. save model of every epoch √
3. reshuffle at every epoch  √
4. if the speed is improved after altering the num_of_work to 40 - *not evident* 
5. if the network is trained better (detection accuracy, the trend of loss) when batch_size=8 (batchs per epoch =1000)

The best performance has reached **0.955**, but the dataset was shuffled firstly before dividing into training set and validation set, which means that **the most objects in validation set were not novel**. 

Results are saved in /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell

In [ ]:
!python train_network.py --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --ds-shuffle --num-workers 40 --random-seed 520

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.90, ds-rotate = 0.00, number of workers = 40, random seed = 520.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 1000, optimizer = adam
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 796
root        : INFO     Validation size: 89
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d-2         [-1, 32, 224, 224]              64
            Conv2d-3         [-1, 64, 112, 112]    

In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_16_iou_0.96 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --augment --ds-shuffle --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_16_iou_0.96
INFO:root:Average evaluation time per image: 104.29836073837711ms
INFO:root:IOU Results: 807/885 = 0.911864


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_16_iou_0.96 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_16_iou_0.96
INFO:root:Average evaluation time per image: 97.35761389220501ms
INFO:root:IOU Results: 777/885 = 0.877966


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_16_iou_0.96 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --random-seed 30

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 89
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_16_iou_0.96
INFO:root:Average evaluation time per image: 105.71914308526542ms
INFO:root:IOU Results: 85/89 = 0.955056


Why do different random-seeds (520, 123, 20, etc) achieve the same evaluation results ?


*   Is the validation results in training process caculated by cross-validation (but why the detection result of the whole set is so low)? 
*   Is the random seed in evaluate.py only change the order of the 89 images but the images themselves is unable to be changed?

*   the evaluate.py results is diffrent from the results of validation in training, **is that caused by reshuffling ?**



In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_29_iou_0.84 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --ds-shuffle --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_29_iou_0.84
INFO:root:Average evaluation time per image: 100.96240232219805ms
INFO:root:IOU Results: 730/885 = 0.824859


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_29_iou_0.84 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 89
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_29_iou_0.84
INFO:root:Average evaluation time per image: 106.66910985882362ms
INFO:root:IOU Results: 81/89 = 0.910112


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_29_iou_0.84 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --random-seed 520

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 89
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_29_iou_0.84
INFO:root:Average evaluation time per image: 104.3187848637613ms
INFO:root:IOU Results: 81/89 = 0.910112


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_26_iou_0.93 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --split 0 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_26_iou_0.93
INFO:root:Average evaluation time per image: 100.01989704067424ms
INFO:root:IOU Results: 798/885 = 0.901695


In [ ]:
!python evaluate.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_26_iou_0.93 --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --iou-eval --random-seed 520

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 89
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210324_0457_training_cornell/epoch_26_iou_0.93
INFO:root:Average evaluation time per image: 105.84091068653578ms
INFO:root:IOU Results: 85/89 = 0.955056


What the hell...?

In [ ]:
!python run_offline.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210402_0701_training_cornell/epoch_19_iou_0.92 --rgb_path /content/cornell_grasp_dataset/06/pcd0601r.png --depth_path /content/cornell_grasp_dataset/06/pcd0601d.tiff --save 1

INFO:root:Loading image...
INFO:root:Loading model...
INFO:root:Done
INFO:root:CUDA detected. Running with GPU acceleration.


In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8 --ds-shuffle --ds-rotate 0.0

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     dropout prob = 0.10, channel size = 16
root        : INFO     split = 0.80, ds-rotate = 0.00, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 16, bacthes per epoch = 448, optimizer = adam
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]           5,200
       BatchNorm2d-2         [-1, 16, 224, 224]              32
            Conv2d-3         [-1, 32, 112, 112]    

*Date: 2nd April*
**5-fold Cross Validation**  
--ds-rotate = 0.0, 0.2, 0.4, 0.6, 0.8  

**Good Performance**  
**1. ds-rotate =0.0, Image Wise, batchsize=16, learning rate =0.002, channel size 16**  
**Achieved 0.938**

saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210402_0701_training_cornell

In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-shuffle --ds-rotate 0.0 --lr=0.002

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 16
root        : INFO     split = 0.80, ds-rotate = 0.00, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 16, bacthes per epoch = 448, optimizer = adam
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]           5,200
       BatchNorm2d-2         [-1, 16, 224, 

**1. ds-rotate =0.0, Object-Wise, batchsize=8, Adam, learning rate =0.001, channel size 32**  
**Achieved 0.927**
logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell




In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.0 --lr=0.001

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.00, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)

# ds-rotate=0.0 without augmentation, Object-Wise, batch size=8, use rgbd,channel size =32

## GCoTNet

In [37]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/GR_ConvNet_Code_GCoTNet')
!pwd

/content/drive/MyDrive/GR_ConvNet_Code
/content/drive/MyDrive/GR_ConvNet_Code_GCoTNet


In [40]:
#!python -m pip install 'git+https://github.com/RealMarco/detectron2.git'
#!git clone https://github.com/RealMarco/detectron2.git
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html  --target /content/drive/MyDrive/GR_ConvNet_Code_GCoTNet

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
  Using cached https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/detectron2-0.5%2Bcu102-cp37-cp37m-linux_x86_64.whl (6.4 MB)
  Using cached Pillow-8.3.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached tensorboard-2.6.0-py3-none-any.whl (5.6 MB)
  Using cached iopath-0.1.8-py3-none-any.whl (19 kB)
  Using cached pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
  Using cached fvcore-0.1.5.post20210924-py3-none-any.whl
  Using cached future-0.18.2-py3-none-any.whl
  Using cached omegaconf-2.1.1-py3-none-any.whl (74 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
  Using cached cloudpickle-2.0.0-py3-none-any.whl (25 kB)
  Using cached black-21.4b2-py3-none-any.whl (130 kB)
  Using cached pydot-1.4.2-py2.py3-none-any.whl (21 kB)
  Using cached hy

In [51]:
# import detectron.detectron2 as detectron2
from detectron2.modeling.backbone.cotnet import CoTBlock ###

ModuleNotFoundError: ignored

**1. gcotnet without augmentation, ds-rotate=0.0, Object-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32**  
**Achived 0**saved at

In [52]:
!python train_network_o.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --network=gcotnet --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.0 --augment 0 --lr=0.001 --epochs 80

INFO:matplotlib.font_manager:generated new fontManager
root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla T4, number of GPUs: 1
root        : INFO     network = gcotnet, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.00, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 80, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210925_1349_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
---------------------

In [57]:
rm -rf /content/drive/MyDrive/GR_ConvNet_Code/logs/210925_0919_training_cornell

In [36]:
cp -r /content/drive/MyDrive/GR_ConvNet_Code/train_network_it.py   /content/drive/MyDrive/GR_ConvNet_Code_GCoTNet/

## GR-ConvNet

**1. without augmentation, ds-rotate=0.0, Object-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32**  
**Achived 0.944**saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1500_training_cornell

In [ ]:
!python train_network_o.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.0 --augment 0 --lr=0.001

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.00, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1500_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
-------------------------------------------------------------

**1. without augmentation, ds-rotate=0.0, Object-Wise, Transfer Learning  batch size=8, use rgbd, SGD 0.001, channel size =32**  
**Achived ** saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210512_0344_training_cornell
perform poor没有达到Adam+SGD优化的效果



In [ ]:
!python train_network_ot.py --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1500_training_cornell/epoch_29_iou_0.94 --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.0 --optim SGD --lr=0.0005 --augment 0 

**1. without augmentation, ds-rotate=0.0, Object-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32 计算更多轮more epochs = 50**  
**Achived 0.949 ** saved at  /content/drive/MyDrive/GR_ConvNet_Code/logs/210512_0447_training_cornell

In [ ]:
!python train_network_o.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.0 --augment 0 --lr=0.001 --epochs 50

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.00, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 50, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210512_0447_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
-------------------------------------------------------------

**visualisation by tensorboard**  
without dropout: /content/drive/MyDrive/GR_ConvNet_Code/logs/210512_0329_training_cornell  
without BN: /content/drive/MyDrive/GR_ConvNet_Code/logs/210513_1000_training_cornell  
original: /content/drive/MyDrive/GR_ConvNet_Code/logs/210512_0447_training_cornell  
/content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1500_training_cornell

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-05-18 03:57:58--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.202.43.88, 52.5.194.13, 54.159.34.239, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.202.43.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.4’

ngrok-stable-linux- 100%[===================>]  13.19M  15.6MB/s    in 0.8s    

2021-05-18 03:57:59 (15.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.4’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [ ]:
LOG_DIR = '/content/drive/MyDrive/GR_ConvNet_Code/logs/210513_1432_training_cornell'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6008 &'.format(LOG_DIR))
#开启ngrok service，绑定port 6006(tensorboard)
get_ipython().system_raw('./ngrok http 6008 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://0ae3a3e79a2e.ngrok.io


In [ ]:
!tensorboard --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs/210513_1432_training_cornell --port 6008 

2021-05-18 04:07:09.326738: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6008/ (Press CTRL+C to quit)
^C


**1. without augmentation, without Batch Normalization in convolutional and transposed convolutional layer, ds-rotate=0.0, Object-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32  epochs = 50**  
**Achived  ** saved at  /content/drive/MyDrive/GR_ConvNet_Code/logs/210513_1000_training_cornell  
grconvnet3_noBN


In [ ]:
!python train_network_o.py --network grconvnet3_nobn --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.0 --augment 0 --lr=0.001 --epochs 50

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3_nobn, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.00, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 50, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210513_1000_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
--------------------------------------------------------

**1. without augmentation, without dropout (dropout probability = 0) ds-rotate=0.0, Object-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32**  
**Achived 0.944 but slower**saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210512_0329_training_cornell



In [ ]:
!python train_network_o.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.0 --augment 0 --lr=0.001 --epochs 50 --dropout-prob 0 

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.00, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.00, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 50, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210512_0329_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
-------------------------------------------------------------

**1. without augmentation, ds-rotate=0.0, Image-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32**  
**Achived 0.910** saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1826_training_cornell


In [ ]:
!python train_network_i.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.0 --augment 0 --lr=0.001 --epochs=50

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.00, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 50, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1826_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
-------------------------------------------------------------

**Evaluation --split 0.8 --ds-rotate 0.0 --iou-eval --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93 **

Why the validation accuracy in evaluation segment (0.915) is different from that in training segment (0.93) ? 

*   Dropout, BN layer, etc produce randomness, and the lack of **net.eval()** may be responsible for it. 
*   random_zoom and random_rotate.





In [ ]:
!python evaluate2.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93 --dataset cornell --dataset-path /content/cornell_dataset --num-workers 8 --split 0.8 --ds-rotate 0.0 --iou-eval

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 177
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93
INFO:root:Average evaluation time per image: 78.81698500638628ms
INFO:root:IOU Results: 162/177 = 0.915254


**Evaluation --split 0.0 --ds-rotate 0.0 --iou-eval --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93 **

In [ ]:
!python evaluate2.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93 --dataset cornell --dataset-path /content/cornell_dataset --num-workers 8 --split 0.0 --iou-eval

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 885
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93
INFO:root:Average evaluation time per image: 77.28449476640776ms
INFO:root:IOU Results: 777/885 = 0.877966


**Do not use augmentation (random zoom and random rotate) in evaluation segment, because this adds randomness which makes outputs unstable.** 

In [ ]:
!python evaluate2.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93 --dataset cornell --dataset-path /content/cornell_dataset --num-workers 8 --split 0.8 --ds-rotate 0.0 --iou-eval --augment

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 177
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0716_training_cornell/epoch_17_iou_0.93
INFO:root:Average evaluation time per image: 84.30235399364753ms
INFO:root:IOU Results: 153/177 = 0.864407


**2.ds-rotate=0.2, Image-Wise, Using original code train_network.py to train the cornell dataset, , batch size=8, use rgbd, Adam 0.001, channel size =32**

**achieved 0.927**
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0837_training_cornell

In [ ]:
!python train_network.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8 --ds-shuffle --ds-rotate 0.2

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Done
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d-2         [-1, 32, 224, 224]              64
            Conv2d-3         [-1, 64, 112, 112]          32,832
       BatchNorm2d-4         [-1, 64, 112, 112]             128
            Conv2d-5          [-1, 128, 56, 56]         131,200
       BatchNorm2d-6          [-1, 128, 56, 56]             256
            Conv2d-7          [-1, 128, 56, 56]         147,584
       BatchNorm2d-8          [-1, 128, 56, 56]             256
            C

In [ ]:
!rm -rf /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1408_training_cornell

**2. without augmentation, ds-rotate=0.2, Image-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32**  
achive 0.927 saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1410_training_cornell

In [ ]:
!python train_network_o.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.2 --augment 0 --lr 0.001 --epochs 30

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.20, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_1410_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
-------------------------------------------------------------

**2. without augmentation, ds-rotate=0.2, Object-Wise, batch size=8, use rgbd, Adam 0.001, channel size =32 epochs=100**  
achive  saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210513_1411_training_cornell

In [ ]:
!python train_network_o.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.2 --augment 0 --lr 0.001 --epochs 100

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.20, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 100, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210513_1411_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
------------------------------------------------------------

**2. ds-rotate =0.2, Object-Wise, batchsize=16, learning rate =0.002, channel size 32**  
**achived 0.864**

saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210403_1609_training_cornell



In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-shuffle --ds-rotate 0.2 --lr=0.002

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.20, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 16, bacthes per epoch = 448, optimizer = adam, learning rate = 0.002000
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2

**Suboptimal performance - Object-Wise**  
[10:41:40] {train_network3.py:243} INFO - network = grconvnet3, use-depth = 1, use-rgb = 1  
[10:41:40] {train_network3.py:244} INFO - dropout prob = 0.10, channel size = 32  
[10:41:40] {train_network3.py:245} INFO - split = 0.80, ds-rotate = 0.20, number of workers = 8, random seed = 123.00  
[10:41:40] {train_network3.py:246} INFO - epochs = 30, **batch size = 8**, bacthes per epoch = 896, optimizer = adam, **learning rate = 0.001000**  
**I haved tried another training, which is set as above, but require a suboptimum result 0.876**  
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210403_1041_training_cornell  



**2. Tranfer Learning ds-rotate =0.2, Object-Wise, batchsize=8, SGD, learning rate =0.0002, channel size 32, train_network_ot.py**  
**Achieved 0.887 in Training, **
****  
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210413_0927_training_cornell

In [ ]:
!python train_network_ot.py --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210403_1041_training_cornell/epoch_06_iou_0.88 --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-rotate 0.2 --optim SGD --lr=0.0002 --augment 1 

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla T4, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1, augment=1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.20, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = SGD, learning rate = 0.000200
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210413_0927_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
    

**Transfer Learning from Jacquard to Cornell**  
original network: /content/drive/MyDrive/jacquard_logs/210412_0327_training_jacquard/epoch_01_iou_0.67




In [ ]:
!python train_network_ot.py --network pretrained_net --pretrained-net-path /content/drive/MyDrive/jacquard_logs/210412_0327_training_jacquard/epoch_01_iou_0.67 --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-rotate 0.2  --lr=0.001 --augment 1 

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1, augment=1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.20, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_0345_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
--------------------------------------------------------

**3. ds-rotate=0.4, Image-Wise, Using original code train_network.py to train the cornell dataset, , batch size=8, use rgbd, Adam 0.001, channel size =32**  
**achieved 0.927**
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_0440_training_cornell  

In [ ]:
!python train_network.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8 --ds-shuffle --ds-rotate 0.4

**3. ds-rotate =0.4, Object-Wise, batchsize=8, Adam, learning rate =0.001, channel size 32**  
**Achieved 0.85**

/content/drive/MyDrive/GR_ConvNet_Code/logs/210409_0226_training_cornell

In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.4 --lr=0.001

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.40, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d

**Transfer Learning**  
train_network_t.py is applied for transfer learning    
**3. ds-rotate =0.4, Object-Wise, batchsize=8, learning rate =0.001, channel size 32**   
**Achieved 0.876**  
Results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210404_1451_training_cornell  


In [ ]:
!python train_network_t.py --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210321_0850_training_cornell/epoch_13_iou_0.97  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.4 --lr=0.001

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.40, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNo

**3. ds-rotate =0.4, Object-Wise, batchsize=16, learning rate =0.002, channel size 16, epoch=100**  
**0.87**
/content/drive/MyDrive/GR_ConvNet_Code/logs/210407_0321_training_cornell

In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-shuffle --ds-rotate 0.4 --lr=0.002 --epochs 100

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 16
root        : INFO     split = 0.80, ds-rotate = 0.40, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 100, batch size = 16, bacthes per epoch = 448, optimizer = adam, learning rate = 0.002000
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]           5,200
       BatchNorm

In [ ]:
!python evaluate_save.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_00_iou_0.97  --dataset cornell --dataset-path /content/cornell_dataset --num-workers 4 --split 0.0 --ds-rotate 0.0 --iou-eval --augment --vis 

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
Traceback (most recent call last):
  File "evaluate_save.py", line 88, in <module>
    include_rgb=args.use_rgb)
  File "/content/drive/My Drive/GR_ConvNet_Code/utils/data/cornell_data.py", line 26, in __init__
    raise FileNotFoundError('No dataset files found. Check path: {}'.format(file_path))
FileNotFoundError: No dataset files found. Check path: /content/ad


**手动学习率衰减, Object-Wise, learning rate =0.001 → 0.0005**  
**0.881 in training validation and 0.915 in evaluation**
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_0259_training_cornell  

In [ ]:
!python train_network_t.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_1556_training_cornell/epoch_17_iou_0.87 --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-shuffle --ds-rotate 0.4 --lr=0.0005 --epochs 30

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 16
root        : INFO     split = 0.80, ds-rotate = 0.40, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 16, bacthes per epoch = 448, optimizer = adam, learning rate = 0.000500
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]           5,200
       BatchN

In [ ]:
!python evaluate2.py --network  /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_1556_training_cornell/epoch_17_iou_0.87  /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_0259_training_cornell/epoch_07_iou_0.88   --dataset cornell --dataset-path /content/cornell_dataset --num-workers 8 --split 0.8 --ds-rotate 0.4 --iou-eval --ds-shuffle

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 177
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_1556_training_cornell/epoch_17_iou_0.87
INFO:root:Average evaluation time per image: 103.66016457983329ms
INFO:root:IOU Results: 155/177 = 0.875706
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_0259_training_cornell/epoch_07_iou_0.88
INFO:root:Average evaluation time per image: 104.33744710717498ms
INFO:root:IOU Results: 162/177 = 0.915254


**3. Tranfer Learning ds-rotate =0.4, Object-Wise, batchsize=16, SGD, learning rate =0.0002, channel size 32, train_network_ot.py**  
**Achieved 0. in Training, **
****  
network  /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_0321_training_cornell/epoch_26_iou_0.87
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210413_0922_training_cornell



In [ ]:
!python train_network_ot.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_0321_training_cornell/epoch_26_iou_0.87 --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-rotate 0.4 --optim SGD --lr=0.0002 --epochs 30 --augment 1

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1, augment=1
root        : INFO     dropout prob = 0.10, channel size = 16
root        : INFO     split = 0.80, ds-rotate = 0.40, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 16, bacthes per epoch = 448, optimizer = SGD, learning rate = 0.000200
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210413_0922_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
--------------------------------------------------------

**手动学习率衰减 Object-Wise learning rate =0.002 → 0.001**  
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_1556_training_cornell  

In [ ]:
!python train_network_t.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_0321_training_cornell/epoch_05_iou_0.85 --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-shuffle --ds-rotate 0.4 --lr=0.001 --epochs 30

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 16
root        : INFO     split = 0.80, ds-rotate = 0.40, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 16, bacthes per epoch = 448, optimizer = adam, learning rate = 0.001000
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]           5,200
       BatchN

In [ ]:
!python train_network_t.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210407_1556_training_cornell/epoch_17_iou_0.87 --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-shuffle --ds-rotate 0.4 --lr=0.0005 --epochs 30

**Adam learning rate =0.0005 → SGD lr= 0.0005**    
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_0724_training_cornell  

In [ ]:
!python train_network_t.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_0259_training_cornell/epoch_07_iou_0.88  --channel-size 16 --num-workers 8 --batch-size 16 --batches-per-epoch 448 --split 0.8  --ds-shuffle --ds-rotate 0.4 --optim SGD --lr=0.0005 --epochs 30

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 16
root        : INFO     split = 0.80, ds-rotate = 0.40, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 16, bacthes per epoch = 448, optimizer = SGD, learning rate = 0.000500
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]           5,200
       BatchNo

**4. ds-rotate =0.6, Object-Wise, batchsize=8, Adam, learning rate =0.001, channel size 32**  
**Achieved 0.921 **

saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_1509_training_cornell  

In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.6 --lr=0.001

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.60, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d

**4. ds-rotate =0.6, Image-Wise, batchsize=8, Adam, learning rate =0.001, channel size 32, train_network4.py**  
**Achieved 0.921**
logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0319_training_cornell

In [ ]:
!python train_network4.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.6 --lr=0.001 --augment 1

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.60, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0319_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        L

**4. Tranfer Learning ds-rotate =0.6, Object-Wise, batchsize=8, SGD, learning rate =0.0001, channel size 32, train_network_t.py**  
**Achieved 0.972 in Training validation, and 0.966 when testing**  
**COOL !**  
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell  


In [ ]:
!python train_network_t.py --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0319_training_cornell/epoch_16_iou_0.92 --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.6  --optim SGD --lr=0.0001 

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.60, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = SGD, learning rate = 0.000100
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNor

**4. Tranfer Learning ds-rotate =0.6, Image-Wise, batchsize=8, SGD, learning rate =0.0001, channel size 32, train_network_it.py**  
**Achieved 0. in Training validation, and 0. when testing**  
**Failed keep 0.921 !**  
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210413_0626_training_cornell


In [ ]:
!python train_network_it.py --network pretrained_net --pretrained-net-path /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0319_training_cornell/epoch_16_iou_0.92 --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-rotate 0.6  --optim SGD --lr=0.0001 --augment 1

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = pretrained_net, use-depth = 1, use-rgb = 1, augment=1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.60, ds-shuffle = 1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = SGD, learning rate = 0.000100
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210413_0626_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
-------------------------------------------------------

**4. Evaluate2 the 4th, Object-Wise ds-rotate=0.6**
**Achievd 0.966**

In [ ]:
!python evaluate2.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_00_iou_0.97 /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_04_iou_0.97 /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_12_iou_0.97 /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_13_iou_0.96 /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_16_iou_0.96 /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_20_iou_0.96 /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_23_iou_0.96  --dataset cornell --dataset-path /content/cornell_dataset --num-workers 8 --split 0.8 --ds-rotate 0.6 --iou-eval

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 177
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_00_iou_0.97
INFO:root:Average evaluation time per image: 108.9630490642483ms
INFO:root:IOU Results: 171/177 = 0.966102
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_04_iou_0.97
INFO:root:Average evaluation time per image: 112.368862507707ms
INFO:root:IOU Results: 166/177 = 0.937853
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_12_iou_0.97
INFO:root:Average evaluation time per image: 111.13976624052403ms
INFO:root:IOU Results: 165/177 = 0.932203
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210412_0930_training_cornell/epoch_13_iou_0.96
INFO:root:Average evaluation time per image: 110.59224807609947m

5. without augmentation image wise

In [ ]:
!python train_network_i.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --num-workers 8 --batch-size 8 --batches-per-epoch 89 --split 0.8  --ds-rotate 0.8 --augment 0 --lr=0.001 --epochs 50

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.80, ds-shuffle=1, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 50, batch size = 8, bacthes per epoch = 89, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210414_2040_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
-------------------------------------------------------------

**5. ds-rotate =0.8, Object-Wise, batchsize=8, Adam, learning rate =0.001, channel size 32**  
**Achieved 0.949, and 0.96 when evaluate the trained net**
需要继续训练

saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_1516_training_cornell  

In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.8 --lr=0.001

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.80, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d

In [ ]:
!python evaluate2.py --network /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_1516_training_cornell/epoch_14_iou_0.95 --dataset cornell --dataset-path /content/cornell_dataset --num-workers 8 --split 0.8 --ds-rotate 0.8 --iou-eval

INFO:root:CUDA detected. Running with GPU acceleration.
INFO:root:Loading Cornell Dataset...
INFO:root:Validation size: 177
INFO:root:Done
INFO:root:
Evaluating model /content/drive/MyDrive/GR_ConvNet_Code/logs/210408_1516_training_cornell/epoch_14_iou_0.95
INFO:root:Average evaluation time per image: 108.31071023887161ms
INFO:root:IOU Results: 170/177 = 0.960452


**5. ds-rotate =0.8, Image-Wise, batchsize=8, Adam, learning rate =0.001, channel size 32, train_network4.py **  
**Achieved 0.898**

logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210411_0857_training_cornell

In [ ]:
!python train_network4.py --dataset cornell --dataset-path /content/cornell_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.8 --lr=0.001 --augment 1

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla T4, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.80, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210411_0857_training_cornell
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type) 

**Poor performance**
**Batch size =64, which is a lot larger than the optimal value.**  
**Ignoring augmentaion also results in the poor performance.**   
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210326_0547_training_cornell

In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs --ds-shuffle --num-workers 0 --batch-size 64 --batches-per-epoch 16  --epochs 50

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.90, ds-rotate = 0.00, number of workers = 0, random seed = 123.00
root        : INFO     epochs = 50, batch size = 64, bacthes per epoch = 16, optimizer = adam
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 796
root        : INFO     Validation size: 89
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d-2         [-1, 32, 224, 224]              64
            Conv2d-3         [-1, 64, 112, 112]      

**Poor performance - Batch size =32, which is larger than the optimal value.**    
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210402_1537_training_cornell  


In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 32 --batches-per-epoch 224 --split 0.8  --ds-shuffle --ds-rotate 0.2

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.20, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 32, bacthes per epoch = 224, optimizer = adam
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d-2         [-1, 32, 224, 

**Poor Performance - Learning rate is assigned to 0.005 (when Batch size =8), which is larger than the optimal value.**    
saved at /content/drive/MyDrive/GR_ConvNet_Code/logs/210403_0307_training_cornell  


In [ ]:
!python train_network3.py --dataset cornell --dataset-path /content/drive/MyDrive/cornell_grasp_dataset --description training_cornell --logdir /content/drive/MyDrive/GR_ConvNet_Code/logs  --channel-size 32 --num-workers 8 --batch-size 8 --batches-per-epoch 896 --split 0.8  --ds-shuffle --ds-rotate 0.2

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.80, ds-rotate = 0.20, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 8, bacthes per epoch = 896, optimizer = adam
root        : INFO     Loading Cornell Dataset...
root        : INFO     Dataset size is 885
root        : INFO     Training size: 708
root        : INFO     Validation size: 177
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]          10,400
       BatchNorm2d-2         [-1, 32, 224, 2

In [ ]:
!unzip -n /content/drive/MyDrive/cornell-grasp.zip -d /content/cornell_dataset

In [ ]:
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_0.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_1.zip -d /content/jacquard_dataset



In [ ]:
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_2.zip -d /content/jacquard_dataset

**Training Jacquard**  
**use train_network3.py to train Jacquard dataset, RGB-D,batchsize=8, Adam, learning rate =0.001, channel size 32, small dataset=jacquard 0,1, shuffling only at every epoch. augment = 0 (random_rotate=False, random_zoom=False)**  

**Achieved**  
saved at /content/drive/MyDrive/jacquard_logs/210409_1221_training_jacquard


In [ ]:
!python train_network3.py --dataset jacquard --dataset-path /content/jacquard_dataset --description training_jacquard --logdir /content/drive/MyDrive/jacquard_logs/ --num-workers 8 --batch-size 8 --batches-per-epoch 1052 --epochs 50 --lr=0.001 --ds-shuffle --augment 0

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla V100-SXM2-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.90, ds-rotate = 0.00, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 50, batch size = 8, bacthes per epoch = 1052, optimizer = adam, learning rate = 0.001000
root        : INFO     logs and results are saved at /content/drive/MyDrive/jacquard_logs/210409_1221_training_jacquard
root        : INFO     Loading Jacquard Dataset...
root        : INFO     Dataset size is 9352
root        : INFO     Training size: 8416
root        : INFO     Validation size: 936
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Lay

In [ ]:
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_0.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_1.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_2.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_3.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_4.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_5.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_6.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_7.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_8.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_9.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_10.zip -d /content/jacquard_dataset
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_11.zip -d /content/jacquard_dataset

In [ ]:
!unzip -n /content/drive/MyDrive/Grasp_Detection_Workplace/Jacquard_Dataset_0.zip -d /content/jacquard_dataset

**Training the Jacquard dataset 0 but augment it  - Object-Wise (train_network3.py)**  
use-depth = 1, use-rgb = 1, channel size =32, batchsize =64, Adam (lr=0.01) --augment 0  
logs and results are saved at /content/drive/MyDrive/jacquard_logs/210410_1542_training_jacquard

In [ ]:
!python train_network3.py --dataset jacquard --dataset-path /content/jacquard_dataset/ --description training_jacquard --logdir /content/drive/MyDrive/jacquard_logs/ --ds-shuffle --num-workers 8 --batch-size 64 --batches-per-epoch 264 --lr 0.01 --augment 1

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=1
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.90, ds-rotate = 0.00, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 64, bacthes per epoch = 264, optimizer = adam, learning rate = 0.010000
root        : INFO     logs and results are saved at /content/drive/MyDrive/jacquard_logs/210410_1542_training_jacquard
root        : INFO     Loading Jacquard Dataset...
root        : INFO     Dataset size is 4646
root        : INFO     Training size: 4181
root        : INFO     Validation size: 465
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        Lay

**Training the whole Jacquard dataset  - Object-Wise (train_network3.py)**  
use-depth = 1, use-rgb = 1, channel size =32, batchsize =64, Adam (lr=0.01) --augment 1  
saved at /content/drive/MyDrive/jacquard_logs/210410_0519_training_jacquard



In [ ]:
!python train_network3.py --dataset jacquard --dataset-path /content/jacquard_dataset/ --description training_jacquard --logdir /content/drive/MyDrive/jacquard_logs/ --ds-shuffle --num-workers 8 --batch-size 64 --batches-per-epoch 767 --lr 0.01 --augment 0

root        : INFO     CUDA detected. Running with GPU acceleration.
root        : INFO     GPU: Tesla P100-PCIE-16GB, number of GPUs: 1
root        : INFO     network = grconvnet3, use-depth = 1, use-rgb = 1, augment=0
root        : INFO     dropout prob = 0.10, channel size = 32
root        : INFO     split = 0.90, ds-rotate = 0.00, number of workers = 8, random seed = 123.00
root        : INFO     epochs = 30, batch size = 64, bacthes per epoch = 767, optimizer = adam, learning rate = 0.010000
root        : INFO     logs and results are saved at /content/drive/MyDrive/jacquard_logs/210410_0519_training_jacquard
root        : INFO     Loading Jacquard Dataset...
root        : INFO     Dataset size is 54485
root        : INFO     Training size: 49036
root        : INFO     Validation size: 5449
root        : INFO     Validation set is prepared
root        : INFO     Loading Network...
root        : INFO     Done
----------------------------------------------------------------
        